# Проект по SQL

<div style="border:solid orange 2px; padding: 20px"> <h7 style="color:black; margin-bottom:20px">Привет! Меня зовут Миша, я буду проводить ревью на этом спринте. 
Мои комментарии и замечания далее по тексту помечены различными цветами:
    
---    
<div class="alert alert-block alert-danger">
Критическое замечание, которые следует исправить.
</div> 
    
---
    
<div class="alert alert-block alert-warning">
Замечание\совет на будущее.
</div>
    
---
    
<div class="alert alert-block alert-success">
Когда всё сделано правильно.
</div>
    
---

Для меня главное, не просто указать на ошибки, а чтобы у тебя, в первую очередь, было понимание как это делается, подсказать в каких то моментах. Поэтому если возникнут вопросы по проекту или вопросы общего характера, то не стесняйся - задавай!) Для того, чтобы мне было легче их найти, то выделяй их пожалуйста с помощью цвета, например вот так - <font color=orange>"вопрос, который не останется незамеченным"</font>
    
Удачи!
    
p.s.: не удаляй мои замечания, если предстоит что то доработать в проекте.</h7>

<div class="alert alert-block alert-success">
<b>Комментарии от ревьюера v2:</b>

Все замечания учтены. Успехов на дальнейших спринтах!
</div>

<div class="alert alert-block alert-danger">
<b>Комментарии от ревьюера:</b>

Итог ревью - ты проделал хорошую работу. Почти все запросы корректные и дают правильный результат. Но есть одно задание, которое тебе предстоит доработать, чтобы я смог принять твой проект. Если возникнут вопросы по моим комментариям, пиши здесь - обязательно отвечу. Буду ждать!
</div>

## <span style="color:blue">Декомпозиция проекта

1. **Описание проекта, его основные цели**


2. **Подключение к базе данных**


3. **Открытие файлов с данными, изучение общей информации**


4. **Анализ данных**
    1. Посчитать, сколько книг вышло после 1 января 2000 года;
    1. Для каждой книги посчитать количество обзоров и среднюю оценку;
    1. Определить издательство, которое выпустило наибольшее число книг толще 50 страниц;
    1. Определить автора с самой высокой средней оценкой книг — учитывать только книги с 50 и более оценками;
    1. Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.
 

5. **Сформулировать предложение для нового продукта.**

## Список источников:
- [Полезные приёмы в работе с PostgreSQL](https://habr.com/ru/post/340460/) 
- [Команды PostgreSQL](https://tproger.ru/translations/useful-postgresql-commands/)

## <span style="color:blue">1. Описание проекта, его основные цели

**Основная задача проекта:** проанализировать базу данных сервиса для чтения книг по подписке. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. После анализа данных сформулировать предложение для нового продукта.

## <span style="color:blue">2. Подключение к базе данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', # пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, # порт подключения
 'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

## <span style="color:blue">3. Открытие файлов с данными, изучение общей информации

In [2]:
# Посмотрим список таблиц в базе данных
query_table_name = "SELECT table_name\
                    FROM information_schema.tables\
                    WHERE table_schema NOT IN ('information_schema','pg_catalog')"  
table_name = pd.io.sql.read_sql(query_table_name, con = engine)
table_name


,table_name
0,orders
1,visits
2,advertisment_costs
3,publishers
4,authors
5,reviews
6,ratings
7,books


Далее изучим интересующие нас таблицы

In [3]:
query_books = "SELECT *\
    FROM books"  
books = pd.io.sql.read_sql(query_books, con = engine)

query_ratings = "SELECT *\
    FROM ratings"  
ratings = pd.io.sql.read_sql(query_ratings, con = engine)

query_authors = "SELECT *\
    FROM authors"  
authors = pd.io.sql.read_sql(query_authors, con = engine)

query_reviews = "SELECT *\
    FROM reviews"  
reviews = pd.io.sql.read_sql(query_reviews, con = engine)

query_publishers = "SELECT *\
    FROM publishers"  
publishers = pd.io.sql.read_sql(query_publishers, con = engine)

In [4]:
# Создадим цыкл который открывает таблицы с данными, 
# описывает общую информацию по ним и проверяет 
# строки на наличие пропусков и дубликатов
for data in [books,\
            ratings,\
            authors,\
            reviews,
            publishers]:
    print('*********************************************')
    print(data.head())
    print()
    print('INFO')
    print(data.info())
    print()
    print('КОЛИЧЕСТВО ПРОПУСКОВ В ТАБЛИЦЕ:')
    print(data.isnull().sum())
    print()
    print('КОЛИЧЕСТВО ДУБЛИКАТОВ В ТАБЛИЦЕ:')
    print(data.duplicated().sum())
    print()

*********************************************
   book_id  author_id                                              title  \
0        1        546                                       'Salem's Lot   
1        2        465                 1 000 Places to See Before You Die   
2        3        407  13 Little Blue Envelopes (Little Blue Envelope...   
3        4         82  1491: New Revelations of the Americas Before C...   
4        5        125                                               1776   

   num_pages publication_date  publisher_id  
0        594       2005-11-01            93  
1        992       2003-05-22           336  
2        322       2010-12-21           135  
3        541       2006-10-10           309  
4        386       2006-07-04           268  

INFO
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
book_id             1000 non-null int64
author_id           1000 non-null int64
title               1000 non-n

Рассмотрев общую информацию видно, что наши данные чистые, пропусков и дубликатов нигде нет, все поля соответствуют своим типам данных.

<div class="alert alert-info"> Привет Миша. Увы, но я не нашел как цыклом проверить наличие пропусков, дубликатов и проверить типы данных всех таблиц на языке SQL (1 запрос 1 таблица), а по одной проверять, слишком громоздко выйдет. </div>

<div class="alert alert-block alert-warning">
<b>Комментарии от ревьюера v2:</b>

Извини) наверное моя формулировка смутила тебя. Я не имел в виду проверки с помощью sql запросов. Надо было вывести просто несколько строк из каждой таблицы и их размерности в идеале)
</div>

<div class="alert alert-block alert-danger">
<b>Комментарии от ревьюера:</b>

А как же проверить выгруженные данные? Посмотреть их содержимое? Добавь этот пункт пожалуйста
</div>

## <span style="color:blue">4. Анализ данных

### Посчитаем, сколько книг вышло после 1 января 2000 года;

In [5]:
query_count_books = "SELECT COUNT(book_id) AS book_cnt\
    FROM books\
    WHERE publication_date > '2000-01-01'"  
count_books = pd.io.sql.read_sql(query_count_books, con = engine)
count_books

,book_cnt
0,819


После первого января 2000 года вышло 819 книг

<div class="alert alert-block alert-success">
<b>Комментарии от ревьюера:</b>

Всё верно. Это правильный ответ
</div>

### Для каждой книги посчитаем среднюю оценку и количество обзоров

In [6]:
# Посчитаем среднюю оценку для каждой книги и количество обзоров
query_ratings_reviews = (
    "SELECT books.title,\
            SUBQ_ratings_reviews.book_id,\
            SUBQ_ratings_reviews.rating_mean,\
            SUBQ_ratings_reviews.review_id_cnt\
     FROM (SELECT SUBQ_ratings.book_id, rating_mean, review_id_cnt\
           FROM (SELECT book_id, AVG(rating) AS rating_mean\
                 FROM ratings\
                 GROUP BY book_id) AS SUBQ_ratings\
           LEFT JOIN (SELECT book_id, COUNT(review_id) AS review_id_cnt\
                     FROM reviews\
                     GROUP BY book_id) AS SUBQ_reviews\
           ON SUBQ_ratings.book_id = SUBQ_reviews.book_id) AS SUBQ_ratings_reviews\
     LEFT JOIN books ON SUBQ_ratings_reviews.book_id = books.book_id\
     ORDER BY rating_mean DESC"
)  
ratings_reviews = pd.io.sql.read_sql(query_ratings_reviews, con = engine)
ratings_reviews

,title,book_id,rating_mean,review_id_cnt
0,Pop Goes the Weasel (Alex Cross #5),518,5.00,2.0
1,The Ghost Map: The Story of London's Most Terr...,732,5.00,2.0
2,In the Hand of the Goddess (Song of the Liones...,347,5.00,2.0
3,Tai-Pan (Asian Saga #2),610,5.00,2.0
4,How to Be a Domestic Goddess: Baking and the A...,330,5.00,1.0
...,...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,915,2.25,3.0
996,Junky,371,2.00,2.0
997,His Excellency: George Washington,316,2.00,2.0
998,Drowning Ruth,202,2.00,3.0


<div class="alert alert-block alert-success">
<b>Комментарии от ревьюера v2:</b>

Отлично. Запрос верный!)
</div>

<div class="alert alert-block alert-danger">
<b>Комментарии от ревьюера:</b>

Отлично. Ты рассчитал средний рейтинг и количество обзоров для каждой книги. Но суть задания заключается, что всё это должно выводиться одним запросов одной таблицей
</div>

### Определим издательства, которые выпустили наибольшее число книг толще 50 страниц

In [7]:
query_publisher_top = "SELECT publishers.publisher, publishers.publisher_id, book_cnt\
    FROM publishers\
    LEFT JOIN (SELECT books.publisher_id, COUNT(books.book_id) AS book_cnt\
               FROM books\
               WHERE num_pages > 50\
               GROUP BY publisher_id) AS SUBQ_num_pages_50\
    ON publishers.publisher_id = SUBQ_num_pages_50.publisher_id\
    WHERE book_cnt IS NOT NULL\
    ORDER BY book_cnt DESC"
publisher_id_top = pd.io.sql.read_sql(query_publisher_top, con = engine)
publisher_id_top.head()

,publisher,publisher_id,book_cnt
0,Penguin Books,212,42
1,Vintage,309,31
2,Grand Central Publishing,116,25
3,Penguin Classics,217,24
4,Bantam,35,19


Определили 5 издательств, которые выпустили наибольшее количество книг толще 50 страниц.

<div class="alert alert-block alert-warning">
<b>Комментарии от ревьюера:</b>

Согласен с результатом. Действительно данное издательство (было бы неплохо вывести имя издательства) является лидером по выпущенным книгам
</div>

### Определим автора с самой высокой средней оценкой книг (учитывая только книги с 50 и более оценками)

In [8]:
query_author_top = "SELECT authors.author, authors.author_id, rating_mean\
    FROM (SELECT author_id, AVG(rating_mean) AS rating_mean\
          FROM books\
          LEFT JOIN (SELECT book_id, COUNT(rating_id) AS rating_cnt, AVG(rating) AS rating_mean\
                     FROM ratings\
                     GROUP BY book_id\
                     HAVING COUNT(rating_id) > 50)\
          AS subquery ON books.book_id = subquery.book_id\
          WHERE rating_mean IS NOT NULL\
          GROUP BY author_id) AS SUB_author_top\
    LEFT JOIN authors ON authors.author_id = SUB_author_top.author_id\
    ORDER BY rating_mean DESC"
author_top = pd.io.sql.read_sql(query_author_top, con = engine)
author_top

,author,author_id,rating_mean
0,J.K. Rowling/Mary GrandPré,236,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,402,4.264151
2,J.R.R. Tolkien,240,4.258446
3,Louisa May Alcott,376,4.192308
4,Rick Riordan,498,4.080645
5,William Golding,621,3.901408
6,J.D. Salinger,235,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,469,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,630,3.787879
9,Dan Brown,106,3.754540


Есть список авторов с самой высокой средней оценкой книг. 

<div class="alert alert-block alert-warning">
<b>Комментарии от ревьюера:</b>

И здесь ты не ошибся. Ты верно отфильтровал книги и рассчитал среднйи рейтинг авторов. Но можно было вывести для наглядности имена авторов
</div>

### Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок

Нужно посчитать количество оценок у пользователей (не меньше 50) и по ним уже посчитать среднее количество обзоров.

In [9]:
query_reviews_count_mean = "SELECT AVG(SUBQ.count) AS reviews_count_mean\
                  FROM (SELECT username, COUNT(review_id)\
                        FROM (SELECT reviews.username, reviews.review_id\
                              FROM reviews\
                              LEFT JOIN (SELECT username, COUNT(rating_id) rating_cnt\
                                         FROM ratings\
                                         GROUP BY username\
                                         HAVING COUNT(rating_id) > 50)\
                              AS subquery_1 ON reviews.username = subquery_1.username\
                              WHERE rating_cnt IS NOT NULL) AS SUBQ_1\
                       GROUP BY SUBQ_1.username) AS SUBQ"
reviews_count_mean = pd.io.sql.read_sql(query_reviews_count_mean, con = engine)
reviews_count_mean

,reviews_count_mean
0,24.333333


Среднее количество обзоров от пользователей 24.

<div class="alert alert-block alert-success">
<b>Комментарии от ревьюера:</b>

И это .. правильный ответ! Ты верно отфильтровал пользователей и рассчитал их показатели.

В итоге, 4\5 с первой попытки - это достойный результат)
</div>

## <span style="color:blue">5. Предложение для нового продукта

Имея в библиотеке 819 книг выпущенных после 1 января 2000 года, нужно определить новые закупки, новые направления по авторам, учитывать оценки читателей. На старте лучше поодержать закупки уже популярных авторов, к которым имеется высокий интерес. Так же не маловажно обратить внимание на активных пользователей у которых большое количество обзоров и высокая активность по оценкам произведений. На эту группу людей стоит сконцентрировать свое внимание, увидеть их предпочтения, желания, негатив.  

<div class="alert alert-block alert-success">
<b>Комментарии от ревьюера:</b>

Здорово, что ты постарался сформулировать предложение для бизнеса. Это очень хорошее качество для аналитика -  не просто рассчитывать и анализировать данные, но и давать рекомендации. Развивай его и дальше)
</div>